# Inserire titolo
Analysis phases:
1.   Principal Component Analysis
2.   Companies clustering: find correct algorithm
3.   Study per-cluster performance of funded vs non-funded companies

Everything can be done with R as well as Python, but the latter is easier.

Before starting, we open the dataset and import the needed libraries.

In [23]:
from pandas import DataFrame, read_excel, read_pickle
from os import path
from numpy import int64

base_path: str = "/content/drive/MyDrive/Progetti/Micelli/"

data: DataFrame

if not path.isfile(f"{base_path}data.pkl"):
  data.to_csv(f"{base_path}data.csv")
  data.to_pickle(f"{base_path}data.pkl")
data = read_pickle(f"{base_path}data.pkl")
data = data.infer_objects()

# We need to cast strs to floats, as some of them are misrepresented
columns_with_floats_as_strs: list[str] = ["totalCost", "ecMaxContribution"]
for column_with_floats_as_strs in columns_with_floats_as_strs:
  data[column_with_floats_as_strs] = data[column_with_floats_as_strs].str.replace(",", ".").astype("float")

# We need to convert dates to their epoch representation
columns_with_datetimes: list[str] = ["startDate", "endDate", "ecSignatureDate"]
for column_with_datetimes in columns_with_datetimes:
  data[column_with_datetimes] = data[column_with_datetimes].values.astype(int64) // 10 ** 9

data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34398 entries, 0 to 34397
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  34398 non-null  int64  
 1   acronym             34398 non-null  object 
 2   status              34398 non-null  object 
 3   title               34398 non-null  object 
 4   startDate           34398 non-null  int64  
 5   endDate             34398 non-null  int64  
 6   totalCost           16909 non-null  float64
 7   ecMaxContribution   16353 non-null  float64
 8   legalBasis          34398 non-null  object 
 9   topics              34398 non-null  object 
 10  ecSignatureDate     34398 non-null  int64  
 11  frameworkProgramme  34398 non-null  object 
 12  masterCall          34398 non-null  object 
 13  subCall             34398 non-null  object 
 14  fundingScheme       34398 non-null  object 
 15  nature              587 non-null    object 
 16  obje

,id,acronym,status,title,startDate,endDate,totalCost,ecMaxContribution,legalBasis,topics,ecSignatureDate,frameworkProgramme,masterCall,subCall,fundingScheme,nature,objective,contentUpdateDate,rcn,grantDoi
0,795119,CARBS,CLOSED,Compositional Approximate Reasoning via Bialge...,1548979200,1580428800,91727.4,91727.4,H2020-EU.1.3.,MSCA-IF-2017,1523577600,H2020,H2020-MSCA-IF-2017,H2020-MSCA-IF-2017,MSCA-IF-EF-ST,NaN,Programming languages with probabilistic featu...,2022-08-17 01:18:43,214376,10.3030/795119
1,815989,CORTI,CLOSED,CO-OPERATIVE ROBOT APPLICATION IN THE TEXTILE ...,1527811200,1543536000,NaN,NaN,H2020-EU.2.3.,EIC-SMEInst-2018-2020,1525824000,H2020,H2020-EIC-SMEInst-2018-2020,H2020-SMEInst-2018-2020-1,SME-1,NaN,Textile industry is one of the largest industr...,2022-08-10 13:01:02,216716,10.3030/815989
2,885845,DAN,CLOSED,"High-performance, kiosk-solution for forensic ...",1569888000,1585612800,NaN,NaN,H2020-EU.2.3.,EIC-SMEInst-2018-2020,1576800000,H2020,H2020-EIC-SMEInst-2018-2020,H2020-SMEInst-2018-2020-1,SME-1,NaN,The Dark Web is a haven for criminals. It has ...,2022-08-16 13:06:24,226908,10.3030/885845
3,880439,SUNSTONE,SIGNED,OMTLAB’s Real time positioning system for smar...,1569888000,1656547200,NaN,NaN,H2020-EU.2.3.,EIC-SMEInst-2018-2020,1569369600,H2020,H2020-EIC-SMEInst-2018-2020,H2020-SMEInst-2018-2020-2,SME-2,NaN,The aim of SUNSTONE is to increase the efficie...,2022-08-24 01:07:23,225366,10.3030/880439
4,851631,EDRA,CLOSED,Hardware-Assisted Decoupled Access Execution o...,1556668800,1604102400,NaN,NaN,H2020-EU.1.2.,FETOPEN-03-2018-2019-2020,1554076800,H2020,H2020-FETOPEN-2018-2020,H2020-FETOPEN-2018-2019-2020-03,CSA,NaN,The FET project “EXTRA” (Exploiting eXascale T...,2022-08-18 14:52:37,223863,10.3030/851631


## Principal Component Analysis
PCA is a dimensionality reduction technique that identifies new uncorrelated variables (principal components) capturing the most significant variance, allowing data simplification and visualization.

In addition, we remove not-explainatory values like `id` and `acronym` etc.

In [27]:
from sklearn.decomposition import FactorAnalysis
from sklearn.preprocessing import StandardScaler

useless_columns: list[str] = ["id", "acronym", "status", "title", "nature", "objective", "contentUpdateDate", "rcn", "grantDoi"]

meaningful_data: DataFrame = data.drop(useless_columns, axis=1)
meaningful_data.info()


AttributeError: ignored